In [1]:
from typing import List
import random


from tqdm import tqdm

from chinese_checkers.reinforcement.experience import Experience, ExperienceMetadata, ExperienceCatalog
from chinese_checkers.reinforcement.dql import DQLAgent, DqlModelValidation
from chinese_checkers.simulation import SimulationCatalog, SimulationMetadata, SimulationData

In [2]:
# import logging
# logging.basicConfig(level=logging.DEBUG)

In [16]:
sim_catalog = SimulationCatalog()
sim_metadata: List[SimulationMetadata] = sim_catalog.list_datasets()
sim_metadata

[SimulationMetadata(player_count=2, board_size=4, max_game_length=1000, winning_player='0', name='bootstrap-simulation', version='v0.0.1'),
 SimulationMetadata(player_count=2, board_size=4, max_game_length=1000, winning_player='3', name='bootstrap-simulation', version='v0.0.1'),
 SimulationMetadata(player_count=2, board_size=4, max_game_length=1000, winning_player='None', name='bad-player-3-bootstrap-simulation', version='v0.0.1'),
 SimulationMetadata(player_count=2, board_size=4, max_game_length=1000, winning_player='0', name='bad-player-3-bootstrap-simulation', version='v0.0.1'),
 SimulationMetadata(player_count=2, board_size=4, max_game_length=1000, winning_player='3', name='bad-player-3-bootstrap-simulation', version='v0.0.1')]

In [17]:
winning_players = ["0", "3"]
player_count = 2

simulations: List[SimulationData] = [
    dataset
    for metadata in sim_metadata
    if metadata.winning_player in winning_players
        and metadata.player_count == player_count
    for dataset in sim_catalog.load_dataset(metadata)
]
print(f"Found {len(simulations)} datasets for winning_players: {winning_players} and player_count: {player_count}.")

Found 2167 datasets for winning_players: ['0', '3'] and player_count: 2.


In [18]:
exp_catalog = ExperienceCatalog()
exp_catalog.list_datasets()

[ExperienceMetadata(player_count='2', board_size='4', max_game_length='1000', winning_player='0', name='bootstrap-simulation', version='v0.0.1', generator_name='v0.0.1', current_player='0'),
 ExperienceMetadata(player_count='2', board_size='4', max_game_length='1000', winning_player='0', name='bootstrap-simulation', version='v0.0.1', generator_name='v0.0.1', current_player='3'),
 ExperienceMetadata(player_count='2', board_size='4', max_game_length='1000', winning_player='3', name='bootstrap-simulation', version='v0.0.1', generator_name='v0.0.1', current_player='0'),
 ExperienceMetadata(player_count='2', board_size='4', max_game_length='1000', winning_player='3', name='bootstrap-simulation', version='v0.0.1', generator_name='v0.0.1', current_player='3'),
 ExperienceMetadata(player_count='2', board_size='4', max_game_length='1000', winning_player='0', name='bad-player-3-bootstrap-simulation', version='v0.0.1', generator_name='v0.0.1', current_player='0'),
 ExperienceMetadata(player_count

In [ ]:
random.shuffle(simulations)
with tqdm(simulations, desc="Generating experiences") as pbar:
    for simulation in pbar:
        experiences = Experience.generate_experiences_from_simulation(
            simulation,
            generator_name="v0.0.1"
        )
        exp_catalog.add_record_list(experiences)
        pbar.set_description(f"Generating experiences (Last generated: {len(experiences)} experiences)")


Generating experiences (Last generated: 341 experiences):  31%|███▏      | 678/2167 [12:33:40<12:40:17, 30.64s/it]    

In [ ]:
exp_metadata: List[ExperienceMetadata] = exp_catalog.list_datasets()
exp_metadata

In [ ]:
experiences: List[Experience] = [
    dataset
    for metadata in exp_metadata
    if metadata.winning_player in winning_players
        and metadata.player_count == str(player_count)
    for dataset in exp_catalog.load_dataset(metadata)
]
print(f"Found {len(experiences)} datasets for winning_players: {winning_players} and player_count: {player_count}.")